### Translating javascript to python code
Uncomment and run cell bellow if you want to automaticaly translate javasccript to python. Put your javascript code in the folder 'gee_javascript'

In [11]:
# from geemap.conversion import *

# # Change js_dir to your own folder containing your Earth Engine JavaScripts,
# # such as js_dir = '/path/to/your/js/folder'
# js_dir = "/mnt/d/SpongeCity/heat_islands_app/gee_javascript"

# # Convert all Earth Engine JavaScripts in a folder recursively to Python scripts.
# js_to_python_dir(in_dir=js_dir, out_dir=js_dir, use_qgis=True)
# print("Python scripts saved at: {}".format(js_dir))

### Deriving Land Surface Temperature from Landsat

Disclaimer: This jupyter notebook includes text and code snippets from google-earth-engine.com tutorial for ADVANCED TOPICS >> HUMAN APPLICATIONS >> Heat islands done by TC Chakraborty

Chapter:      A1.5 Heat Islands
Checkpoint:   A15b
Author:       TC Chakraborty

Landsat's thermal bands use the thermal infrared longwave radiation to measure the top-of-atmosphere brightness temperature.(Reiners et al. 2023) Brightness temperature is the temperature equivalent of the infrared radiation escaping the top of the atmosphere, assuming the Earth to be a black body. It is not the same as the land surface temperature (LST), which requires accounting for atmospheric absorption and re-emission, as well as the emissivity of the land surface. The surface emissivity (ε) of a material is the effectiveness with which it can emit thermal radiation compared to a black body at the same temperature and can range from 0 (for a perfect reflector) to 1 (for a perfect absorber and emitter). Since the thermal radiation captured by satellites is a function of both LST and ε, you need to accurately prescribe or estimate ε to get to the correct LST. One way to derive pixel-level emissivity is as a function of the vegetation fraction of the pixel (Malakar et al. 2018). For this, we are calculating the Normalized Difference Vegetation Index (NDVI) from the Landsat surface reflectance data.

It's important to note that Landsat is a polar-orbiting satelllite and it can only provide instantaneous measurements, which do not represent the daily progress of the LST. Thus it is very important to keep in mind the time of image acquisition which might not correspond to the hottest time of the day. Furthurmore, since Landsat images are acquired once every 16 days, this means that the hottest day of the year might nor be captured. 

In [12]:
import geemap.foliumap as geemap
import ee

In [13]:
# ee.Authenticate()
ee.Initialize()

### Imports and user defined variables

In [14]:
# Import areas of interest
plovdiv = ee.FeatureCollection("projects/ee-nikolova100yana/assets/SpongeCity/02_Plovdiv")
pecs = ee.FeatureCollection("projects/ee-nikolova100yana/assets/SpongeCity/03_Pecs")
salzburg = ee.FeatureCollection("projects/ee-nikolova100yana/assets/SpongeCity/04_Salzburg")
chisinau = ee.FeatureCollection("projects/ee-nikolova100yana/assets/SpongeCity/01_ChisinauMunicipality")
# ---------------------and more TO DO

# Area of analyses
regionInt = salzburg

# Period of analyses
start = '2014-01-01'
end = '2019-01-01'


### Functions and analyses

In [15]:
# The function's content was adapted from google-earth-engine.com
# Chapter:      A1.5 Heat Islands
# Checkpoint:   A15b
# Author:       TC Chakraborty

# Add layer and zoom to it in the map
m = geemap.Map(basemap='HYBRID')
m.centerObject(regionInt,12)
#print("regionInt",regionInt.getInfo())

# Create a summer filter. We want to focus on only summertime heat islands  so we will use only
# images from June 1 (day 152) to August 31 (day 243) in each year
sumFilter = ee.Filter.dayOfYear(152, 243)

# Generate a water mask. The high specific heat capacity of water would affect land surface teperature
water = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('occurrence')
notWater = water.mask().Not()

In [16]:
# ------------------------------------------------------------------------------
# LANDSAT SECTION
#--------------------------------------------------------------------------------
# Function to filter out cloudy pixels.
def cloudMask(cloudyScene):
    # Add a cloud score band to the image.
    scored = ee.Algorithms.Landsat.simpleCloudScore(cloudyScene)

    # Create an image mask from the cloud score band and specify threshold.
    mask = scored.select(['cloud']).lte(10)

    # Apply the mask to the original image and return the masked image.
    return cloudyScene.updateMask(mask)


# Load the collection, apply cloud mask, and filter to date and region of interest.
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
.filterBounds(regionInt) \
.filterDate(start, end) \
.filter(sumFilter) \
.map(cloudMask)

# print('Landsat collection', col.getInfo())

# Generate median composite.
image = col.median()

# Select thermal band 10 (with brightness temperature).
thermal = image.select('B10') \
.clip(regionInt) \
.updateMask(notWater)


# Calculate Normalized Difference Vegetation Index (NDVI)
# from Landsat surface reflectance.
ndvi = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterBounds(regionInt) \
.filterDate('2014-01-01', '2019-01-01') \
.filter(sumFilter) \
.median() \
.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') \
.clip(regionInt) \
.updateMask(notWater)


# Find the minimum and maximum of NDVI.  Combine the reducers
# for efficiency (single pass over the data).
minMax = ndvi.reduceRegion(
reducer=ee.Reducer.min().combine(
reducer2=ee.Reducer.max(),
sharedInputs=True
),
geometry=regionInt,
scale=30,
maxPixels=1e9
)
print('minMax', minMax.getInfo())

min = ee.Number(minMax.get('NDVI_min'))
max = ee.Number(minMax.get('NDVI_max'))

# Calculate fractional vegetation.
fv = ndvi.subtract(min).divide(max.subtract(min)).rename('FV')


# Emissivity calculations.
a = ee.Number(0.004)
b = ee.Number(0.986)
em = fv.multiply(a).add(b).rename('EMM').updateMask(notWater)


# Calculate LST from emissivity and brightness temperature.
lstLandsat = thermal.expression(
'(Tb/(1 + (0.001145* (Tb / 1.438))*log(Ep)))-273.15', {
    'Tb': thermal.select('B10'),
    'Ep': em.select('EMM')
}).updateMask(notWater)

# 1.1. Selection of maps to be desplayed

# m.addLayer(regionInt, {}, 'City boundary')

# Landsat brightness temperature
#m.addLayer(thermal, { min: 295, max: 310, palette: ['blue', 'white', 'red']},'Landsat_BT')

# Normalized Vegetation Index
m.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green', 'darkgreen']}, 'NDVI')

# Fractional vegetaion
#m.addLayer(fv, {min: 0, max: 1, palette: ['blue', 'white', 'green']}, 'Fractional vegetation')

# Emissivity
#m.addLayer(em, { min: 0.98, max: 0.99, palette: ['blue', 'white', 'green']},'EMM')

# Land Surface Temperature
m.addLayer(lstLandsat, {'min': 25, 'max': 35, 'palette': ['blue', 'white', 'red']},'Land Surface Temperature')


#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------
m


minMax {'NDVI_max': 0.5235740870448958, 'NDVI_min': -0.10136235921894553}


### Module to obtain time of image acquisition for the chosen area of interest

Our image collection is filtered for the summer period between June 1 (day 152) to August 31 (day 243) in each year.
The Daylight Saving Time (DST) period in Europe runs from the end of March to the end of October every year. Thus time zones are calculated based on the DST time.

For more info on european time zones check:
https://www.timeanddate.com/time/europe/



In [31]:
from datetime import datetime
import pytz

# Get the first image of the filtered collection for the corresponding region
# Since acquisition times in a certain region differ only by second, we will use only the first image
# to get a representative time for the whole image collection

first_img = col.first()
img_time = (first_img.get('SCENE_CENTER_TIME')).getInfo()[:5] #HH:MM
img_date = (first_img.get('DATE_ACQUIRED')).getInfo().replace("-", ":") #YYYY:MM:DD
img_datetime = img_date +":" + img_time

# Convert it to datetime object
time = datetime.strptime(img_datetime, "%Y:%m:%d:%H:%M")

# Assign GMT (UTC) timezone
gmt_time = time.replace(tzinfo=pytz.utc)

# Convert the time according to the time zone of the area of interest
# Define different timezones
cet_timezone = pytz.timezone('Europe/Berlin')  # CET (UTC+1)

# find out in which time zone is the chosen aoi locatied

if regionInt == salzburg:
    aoi_timezone = cet_timezone
    # and so on, for the rest areas if interest

# Convert to CET
aoi_time = gmt_time.astimezone(aoi_timezone)

aoi_time = aoi_time.strftime("%H:%M")
print ('Time:', aoi_time) # return this in function



img_datetime 2014:06:12:09:51
TIME 2014-06-12 09:51:00
UTC
Time Zone Europe/Berlin
Time: 11:51
